# Analysis and Visualization of Complex Agro-Environmental Data
---
## Interactive visualization in python with `Bokeh`, `Plotly`, `Dash` and `Altair`

### 1. Bokeh

`Bokeh` is a Python's module for interactive da visualizations. The plots are created by stacking layers on top of each other. The first step is to create an empty figure, to which elements are added in layers. These elements are known as glyphs, which can be anything from lines to bars to circles. Attached to each glyph are properties such as color, size and coordinates.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### 1.1 Data preparation

Download 2 datasets: (1) CO2 emissions per person per year per country and (2) GDP per year per country:

In [2]:
url_co2 = 'https://raw.githubusercontent.com/TrainingByPackt/Interactive-Data-Visualization-with-Python/master/datasets/co2.csv'
co2 = pd.read_csv(url_co2)
url_gm = 'https://raw.githubusercontent.com/TrainingByPackt/Interactive-Data-Visualization-with-Python/master/datasets/gapminder.csv'
gm = pd.read_csv(url_gm)

Transform the dataset - intersect datasets

In [ ]:
# Drop duplicates in gm
df_gm = gm[['Country', 'region']].drop_duplicates()
# Combine the 2 datasets (merge by country)
df_w_regions = pd.merge(co2, df_gm, left_on ='country', right_on ='Country', how ='inner') # intersection of both keys, keep order from 'left'
# Drop one Country column
df_w_regions = df_w_regions.drop('Country', axis='columns')
df_w_regions.head()

Transform the dataset - stack by year

In [ ]:
# change the format of the DataFrame into one that has identifier variables of our choice (here, country and region).
new_co2 = pd.melt(df_w_regions, id_vars=['country', 'region'])
columns = ['country', 'region', 'year', 'co2']
new_co2.columns = columns
# Select data from 1964 onwards, sort by country and then year
df_co2 = new_co2[new_co2['year'].astype('int64') > 1963]
df_co2 = df_co2.sort_values(by=['country', 'year'])
df_co2['year'] = df_co2['year'].astype('int64')
df_co2.head()


Create similar table for GDP per year per country

In [ ]:
df_gdp = gm[['Country', 'Year', 'gdp']]
df_gdp.columns = ['country', 'year', 'gdp']
df_gdp.head()

Merge datasets

In [ ]:
data = pd.merge(df_co2, df_gdp, on=['country', 'year'], how='left')
data = data.dropna()
data.head()

#### 1.2 Running Bokeh

Import Bokeh and functions

In [8]:
from bokeh.io import curdoc, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, Slider
from bokeh.palettes import Spectral6
from bokeh.layouts import column, row

#### Prepare base static plot

In [9]:
# load BokehJS - enables the plot to be displayed within the notebook
output_notebook()

Loading BokehJS ...

In [24]:
# create list of regions - to color the datapoints based on the region
regions_list = data.region.unique().tolist()
# assign colors to each region
color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)

In [25]:
# make a data source for the plot
source = ColumnDataSource(data={
    'x': data.gdp[data['year'] == 1964],
    'y': data.co2[data['year'] == 1964],
    'country': data.country[data['year'] == 1964],
    'region': data.region[data['year'] == 1964],
})

In [26]:
# Save the minimum and maximum values of the gdp column: xmin, xmax
xmin, xmax = min(data.gdp), max(data.gdp)

# Save the minimum and maximum values of the co2 column: ymin, ymax
ymin, ymax = min(data.co2), max(data.co2)

In [27]:
# Create the figure: plot
plot = figure(title='CO2 Emissions vs GDP in 1964', 
              height=600, width=1000,
              x_range=(xmin, xmax),
              y_range=(ymin, ymax), y_axis_type='log')

In [28]:
# Add circle glyphs to the plot
plot.circle(x='x', y='y', fill_alpha=0.8, source=source, legend_field='region',
            color=dict(field='region', transform=color_mapper),
            size=7)

GlyphRenderer(id='p2042', ...)

In [ ]:
# Set the legend.location attribute of the plot
plot.legend.location = 'bottom_right'

# Set the x-axis label
plot.xaxis.axis_label = 'Income Per Person'

# Set the y-axis label
plot.yaxis.axis_label = 'CO2 Emissions (tons per person)'
show(plot)

#### Adding a hover tool

In [ ]:
# Create a HoverTool - will allow the user to hover above a datapoint to see the name of the country, CO2 emissions nd GDP
hover = HoverTool(tooltips=[('Country', '@country'), ('GDP', '@x'), ('CO2 Emission', '@y')])

# Add the HoverTool to the plot
plot.add_tools(hover)

show(plot)

#### Adding a slider to the static plot

In [ ]:
# Make a slider object: slider
# Set the start as the 1st year and the end as the last year in the year column
# Set step as 1 and the value as the minimum value of the year column
slider = Slider(start=min(data.year), end=max(data.year), step=1, value=min(data.year), title='Year')

# create function that will update the plot every time the silider is moved
def update_plot(attr, old, new):
    # set the `yr` name to `slider.value` and `source.data = new_data`
    yr = slider.value
    new_data = {
        'x': data.gdp[data['year'] == yr],
        'y': data.co2[data['year'] == yr],
        'country': data.country[data['year'] == yr],
        'region': data.region[data['year'] == yr],
    }
    source.data = new_data
    plot.title.text = 'CO2 Emissions vs GDP in %d' % yr

# Attach the callback to the 'value' property of slider
slider.on_change('value', update_plot)
layout = column(slider, plot)
curdoc().add_root(layout)

show(plot)

# ERROR!!! - no slider is shown

#### Creating a standalone HTML file

In [ ]:
from bokeh.plotting import figure, output_file, save

output_file(filename="custom_filename.html", title="Static HTML file")

save(plot, "bokeh_plot.html") # save html plot

show(plot) # show plot in the web browser

#### Slider example from https://www.geeksforgeeks.org/add-interactive-slider-to-bokeh-plots/

In [ ]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.plotting import figure, output_file, show
import numpy as np
  
x2 = np.linspace(0, 10, 500)
y2 = np.sin(x2)
  
source2 = ColumnDataSource(data=dict(x=x2, y=y2))
  
# Create plots and widgets
plot = figure()
  
plot.line('x', 'y', source=source2, line_width=3, line_alpha=0.5)
  
# Create Slider object
slider2 = Slider(start=0, end=6, value=2,
                step=0.2, title='Number of points')
  
# Adding callback code
callback = CustomJS(args=dict(source=source2, val=slider2),
                    code="""
    const data = source.data;
    const freq = val.value;
    const x = data['x'];
    const y = data['y'];
   for (var i = 0; i < x.length; i++) {
        y[i] = Math.sin(freq*x[i]);
    }
      
    source.change.emit();
""")
  
slider2.js_on_change('value', callback)
  
# Arrange plots and widgets in layouts
layout2 = column(slider2, plot)
  
output_file('exam.html')
  
show(layout2)

### 2. Plotly

`plotly` is a very popular Python module used to create interactive data visualizations. It is a JSON-based plotting tool, and so every plot is defined by 2 JSON objects - data and layout. 

A simplified and more user friendly version of `plotly` is `plotly express` which is provides a high-level wrapper around the base `plotly code, resulting in a minimized syntax abd commands.

#### Creating an interactive scatter plot

We will use the same dataset created for Bokeh in the previous example.

In [18]:
# Save the minimum and maximum values of the gdp column: xmin, xmax
xmin, xmax = min(data.gdp), max(data.gdp)
# Save the minimum and maximum values of the co2 column: ymin, ymax
ymin, ymax = min(data.co2), max(data.co2)

In [ ]:
import pandas as pd
import plotly.express as px

fig = px.scatter(data, x="gdp", y="co2", animation_frame="year", animation_group="country",
           color="region", hover_name="country", facet_col="region", width=1579, height=400,
           log_x=True, size_max=45, range_x=[xmin,xmax], range_y=[ymin,ymax])

fig.show()

In [ ]:
# Aggregate in a single plot (by removing facet_col="region") and add rug and boxplot.
fig = px.scatter(data, x="gdp", y="co2", animation_frame="year", 
        color="region", hover_name="country", width=1000, height=600,
        size_max=45, range_x=[xmin,xmax], range_y=[ymin,ymax], 
        marginal_y = 'box', marginal_x = 'rug') # add a boxplot in the left side and rug plot on top

fig.show()

In [ ]:
# N0w using a density contour plot instead of a scatter plot
fig = px.density_contour(data, x="gdp", y="co2", animation_frame="year", 
        color="region", hover_name="country", width=1000, height=600,
        range_x=[xmin,xmax], range_y=[ymin,ymax], 
        marginal_y = 'box', marginal_x = 'rug') # add a boxplot in the left side and rug plot on top

fig.show()

In [ ]:
fig = px.scatter(data, x="year", y="co2", color="region", hover_name="country", width=1000, height=500,
           size_max=45, marginal_y = 'box')

fig.show()

#### Visualizing an output of Principal Component Analysis

##### Example using the `Iris` dataset

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
# pairwise scatter plots
df = px.data.iris()
features = ["sepal_width", "sepal_length", "petal_width", "petal_length"]

fig = px.scatter_matrix(
    df,
    dimensions=features,
    color="species",
    width=800, height=700
)

fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
# PCA plots
pca = PCA()
components = pca.fit_transform(df[features])
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    color=df["species"],
    width=800, height=700
)
fig.update_traces(diagonal_visible=False)

fig.show()

##### Example using the `winequality` dataset

In [ ]:
from sklearn.preprocessing import StandardScaler

df_wine = pd.read_csv('winequality_red.csv')
df_wine2 = df_wine.iloc[:, 0:11]
wine_scaled = StandardScaler().fit_transform(df_wine2)
df_scaled = pd.DataFrame(data=wine_scaled, columns=df_wine2.columns)


pca = PCA()
components = pca.fit_transform(df_scaled)
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(5), # Try to change the number of PC's (from 2 to 11, in this case)
    color=df_wine["quality"],
    width=1000, height=900
)
fig.update_traces(diagonal_visible=False)

fig.show()

### 3. Dash

Python's `dash` module offers a framework for building interactive data visualization interfaces. `dash` helps to build interactive web applications and dashboards to visualize data without requiring advanced web development knowledge.

Bellow you can find a example of a `dash` interactive plot to visualize PCA plots with a user defined number of components. To run the app, copy the code into a new file named pca-visualization.py and type into your terminal the following code:

> python pca-visualization.py

Then, go to the http link by using 'ctr... + mouse click'.

To exit: 'ctr + c'

#### PCA with `Iris` Dataset

In [ ]:
# Import modules
# import Dash, dcc (stands for Dash Core Components - this module includes a Graph component called dcc.Graph, 
# which is used to render interactive graphs amd dcc.slider to render an interactive slider).
# We also import sklearn.decomposition.PCA to run a PCA, the plotly.express library to build the interactive graphs, 
# and pandas to work with DataFrames.

from dash import Dash, dcc, html, Input, Output
from sklearn.decomposition import PCA
import plotly.express as px
import pandas as pd

# Initialize the app
# This line is known as the Dash constructor and is responsible for initializing your app. 
# It is almost always the same for any Dash app you create.
app = Dash(__name__)

# App layout
# The app layout represents the app components that will be displayed in the web browser, 
# normally contained within a html.Div.
app.layout = html.Div([
    html.H4("Visualization of PCA's explained variance", style={'textAlign':'center'}),
    dcc.Graph(id="pca-visualization-x-graph"),
    html.P("Number of components:"),
    dcc.Slider(
        id='pca-visualization-x-slider',
        min=2, max=4, value=2, step=1)
])

# Add controls to build the interaction
# The inputs and outputs of our app are the properties of a particular component. 
# The output is the figure property of the component with the ID "pca-visualization-x-graph"
# THe input is the value property of the component that has the ID "pca-visualization-x-slider".
# The callback function's argument 'n_components' refers to the component property of the input. 
# We build PCA plots inside the callback function, assigning the chosen value in the slider. 
# This means that every time the user selects the number of components with the slider, the figure is rebuilt
# to add more or less components
# Finally, we return the scatter plots at the end of the function. 
# This assigns the plots to the figure property of the dcc.Graph, thus displaying the figure in the app.
@app.callback(
    Output(component_id="pca-visualization-x-graph", component_property="figure"), 
    Input(component_id="pca-visualization-x-slider", component_property="value"))

def run_and_plot(n_components):
    df = px.data.iris().iloc[:,0:4]
    pca = PCA(n_components=n_components) # defines the number of components in the PCA
    components = pca.fit_transform(df) # fits a PCA
    var = pca.explained_variance_ratio_.sum() * 100 # % of explained variance by each PC
    labels = {str(i): f"PC {i+1}" for i in range(n_components)} # PC labels
    labels['color'] = 'species'
    fig = px.scatter_matrix(
        components,
        color=px.data.iris()["species"],
        dimensions=range(n_components),
        labels=labels,
        title=f'Total Explained Variance: {var:.2f}%')
    fig.update_traces(diagonal_visible=False)
    return fig

# Run the app - These lines are for running your app, and they are almost always the same for any Dash app you create.
if __name__ == "__main__":
    app.run_server(debug=True)

#### PCA with `Wine` Dataset

In [ ]:
# ATENTION: THIS CODE DOES NOT WORK IN JUPYTER NOTEBOOK! 
# Need to copy-paste to a *.py file and run in command line (see above).

# Import modules
# import Dash, dcc (stands for Dash Core Components - this module includes a Graph component called dcc.Graph, 
# which is used to render interactive graphs amd dcc.slider to render an interactive slider).
# We also import sklearn.decomposition.PCA to run a PCA, the plotly.express library to build the interactive graphs, 
# and pandas to work with DataFrames.

from dash import Dash, dcc, html, Input, Output
from sklearn.decomposition import PCA
import plotly.express as px
import pandas as pd
from sklearn.preprocessing import StandardScaler

df_wine = pd.read_csv('winequality_red.csv')

# Initialize the app
# This line is known as the Dash constructor and is responsible for initializing your app. 
# It is almost always the same for any Dash app you create.
app = Dash(__name__)

# App layout
# The app layout represents the app components that will be displayed in the web browser, 
# normally contained within a html.Div.
app.layout = html.Div([
    html.H1("Visualization of PCA's explained variance", style={'textAlign':'center'}),
    dcc.Graph(id="pca-visualization-x-graph"),
    html.P("Number of components:"),
    dcc.Slider(
        id='pca-visualization-x-slider',
        min=2, max=11, value=2, step=1)
])

# Add controls to build the interaction
# The inputs and outputs of our app are the properties of a particular component. 
# The output is the figure property of the component with the ID "pca-visualization-x-graph"
# THe input is the value property of the component that has the ID "pca-visualization-x-slider".
# The callback function's argument 'n_components' refers to the component property of the input. 
# We build PCA plots inside the callback function, assigning the chosen value in the slider. 
# This means that every time the user selects the number of components with the slider, the figure is rebuilt
# to add more or less components
# Finally, we return the scatter plots at the end of the function. 
# This assigns the plots to the figure property of the dcc.Graph, thus displaying the figure in the app.
@app.callback(
    Output(component_id="pca-visualization-x-graph", component_property="figure"), 
    Input(component_id="pca-visualization-x-slider", component_property="value"))

def run_and_plot(n_components):
    df_wine = pd.read_csv('winequality_red.csv')
    df_wine2 = df_wine.iloc[:, 0:11]
    wine_scaled = StandardScaler().fit_transform(df_wine2)
    df = pd.DataFrame(data=wine_scaled, columns=df_wine2.columns)
    pca = PCA(n_components=n_components) # defines the number of components in the PCA
    components = pca.fit_transform(df) # fits a PCA
    var = pca.explained_variance_ratio_.sum() * 100 # % of explained variance by each PC
    labels = {str(i): f"PC {i+1}" for i in range(n_components)} # PC labels
    labels['color'] = 'quality'
    fig = px.scatter_matrix(
        components,
        color=df_wine['quality'],
        dimensions=range(n_components),
        labels=labels,
        title=f'Total Explained Variance: {var:.2f}%',
        width=1400, height=1300
        )
    fig.update_traces(diagonal_visible=False)
    return fig

# Run the app - These lines are for running your app, and they are almost always the same for any Dash app you create.
if __name__ == "__main__":
    app.run_server(debug=True)

#### Variation of human population per country 1959-2007

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv')

app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Human Population Size', style={'textAlign':'center'}),
    dcc.Dropdown(df.country.unique(), 'Portugal', id='dropdown-selection'), # dropdown menu
    dcc.Graph(id='graph-content') 
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = df[df.country==value]
    return px.line(dff, x='year', y='pop') # Line graph

if __name__ == '__main__':
    app.run_server(debug=True)

### 4. Altair

`Altair` is a module designed specially to generating interactive plots. This module has some specific advantages over more multi-specific purpose libraries such as `plotly`, as ilustrated in the following examples.

We will use the Happy Planet Index (HPI) http://happyplanetindex.org/ dataset to ilustrate some potentialities of `altair`to generate scatter and bar plots with a variety of interactive elements. The dataset shows *where people are using ecological resources more efficiently to live long, happy lives*.

Import the necessary modules or functions:

In [87]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt

Download and check the dataset. Each line (datapoint) will correspond to a country.

In [ ]:
#Download the data from Github repo 
hpi_url = "https://raw.githubusercontent.com/TrainingByPackt/Interactive-Data-Visualization-with-Python/master/datasets/hpi_data_countries.tsv"

# Once downloaded, read it into a dataframe using pandas
hpi_df = pd.read_csv(hpi_url, sep='\t')
hpi_df.head()

Plot a static scatter plot with `matplotlib`

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax = sns.scatterplot(x='Wellbeing (0-10)', y='Happy Planet Index', hue='Region', data=hpi_df)
plt.show()

Add `zooming`

In [ ]:
alt.Chart(hpi_df).mark_circle().encode( # mark_circle() denote datapoints using filled circles (use mark_points for empty circles)
x='Wellbeing (0-10):Q', # with the encode function you specify the features on the x and y axes.
y='Happy Planet Index:Q', # Q denotes quantitative features (helps altair to identify the type of features)
color='Region:N', # N denotes nominal features
).interactive() # makes the plot interactive for zooming (e.g. using the mouse wheel)

Add `hover`

In [ ]:
selected_area = alt.selection_interval()
alt.Chart(hpi_df).mark_circle().encode(
x='Wellbeing (0-10):Q',
y='Happy Planet Index:Q',
color='Region:N',
tooltip= ['Country', 'Region', 'Wellbeing (0-10)', 'Happy Planet Index', 'Life Expectancy (years)',] # add tooltip with the selected information
)

`Hover` and `zooming`

In [ ]:
selected_area = alt.selection_interval()
alt.Chart(hpi_df).mark_circle().encode(
x='Wellbeing (0-10):Q',
y='Happy Planet Index:Q',
color='Region:N',
tooltip= ['Country', 'Region', 'Wellbeing (0-10)', 'Happy Planet Index', 'Life Expectancy (years)',] # add tooltip with the selected information
).interactive()

`Select` and `highlight` functionality on a scatter plot

In [ ]:
selected_area = alt.selection_interval() # define the selected area variable to store the selection interval

alt.Chart(hpi_df).mark_circle().encode(
x='Wellbeing (0-10):Q',
y='Happy Planet Index:Q',
color=alt.condition(selected_area, 'Region', alt. # condition to retain color only in points inside the selected area (try to click and drag to select a region)
value('lightgray')),
tooltip= ['Country', 'Region', 'Wellbeing (0-10)', 'Happy Planet Index', 'Life Expectancy (years)',]
).add_params(selected_area)

Add hover, tooltip, select and highlight across multiple charts

In [ ]:
# hover and tooltip across multiple charts
selected_area = alt.selection_interval()

chart = alt.Chart(hpi_df).mark_circle().encode(
y='Happy Planet Index',
color=alt.condition(selected_area, 'Region', alt.
value('lightgray'))
).add_params(selected_area)
chart1 = chart.encode(x='Wellbeing (0-10)')
chart2 = chart.encode(x='Life Expectancy (years)')
chart1 | chart2

Selection based on values of a feature using a `dropdown menu`

In [ ]:
selected_area = alt.selection_interval()

input_dropdown = alt.binding_select(options=list(set(hpi_df.Region)))
selected_points = alt.selection_single(fields=['Region'], bind=input_dropdown, name='Select')
color = alt.condition(selected_points,
alt.Color('Region:N'),
alt.value('lightgray'))
alt.Chart(hpi_df).mark_circle().encode(
x='Wellbeing (0-10):Q',
y='Happy Planet Index:Q',
color=color,
tooltip='Region:N'
).add_params(selected_points)

Barplot with line representing the mean of the selected bars

In [ ]:
selected_bars = alt.selection_interval(encodings=['x'])

bars = alt.Chart(hpi_df).mark_bar().encode(
x='Region:N',
y='mean(Happy Planet Index):Q',
opacity=alt.condition(selected_bars, alt.OpacityValue(1), alt.
OpacityValue(0.7)),
).properties(width=400).add_params(selected_bars)

line = alt.Chart(hpi_df).mark_rule(color='firebrick').encode( # draw a line representing the mean "Happy Planet Index"
y='mean(Happy Planet Index):Q',
size=alt.SizeValue(3) # set line thickness
).transform_filter(selected_bars) # condition to only consider the mean of the selected bars
bars + line

Heatplot woth interactive zooming

In [ ]:
alt.Chart(hpi_df).mark_rect().encode(
alt.X('Happy Planet Index:Q', bin=True),
alt.Y('Wellbeing (0-10):Q', bin=True),
alt.Color('count()',
scale=alt.Scale(scheme='greenblue'),
legend=alt.Legend(title='Total Countries')
)
).interactive()

Dinamically linking a bar plot and a heatmap.

Static plots

In [ ]:
#Merge the code to place the bar chart and heatmap next to each other.
bars = alt.Chart(hpi_df).mark_bar().encode(
    x='Region:N',
    y='count():Q',
).properties(width=350)

heatmap = alt.Chart(hpi_df).mark_rect().encode(
    alt.X('Wellbeing (0-10):Q', bin=True),
    alt.Y('Life Expectancy (years):Q', bin=True),
    alt.Color('count()',
        scale=alt.Scale(scheme='greenblue'),
        legend=alt.Legend(title='Total Countries')
    )
).properties(width=350)

bars | heatmap

Dynamic plots

In [181]:
# Heatplot
selected_region = alt.selection(type="single", encodings=['x'])

heatmap = alt.Chart(hpi_df).mark_rect().encode(
    alt.X('Wellbeing (0-10):Q', bin=True),
    alt.Y('Life Expectancy (years):Q', bin=True),
    alt.Color('count()',
        scale=alt.Scale(scheme='greenblue'),
        legend=alt.Legend(title='Total Countries')
    )
).properties(width=350)

In [182]:
# Place circles in the heatplot denoting the number of samples 
circles = heatmap.mark_point().encode(
    alt.ColorValue('grey'),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection')
    )
).transform_filter(selected_region)

After the following code try to click on top on each bar.

In [ ]:
#Place the bars
bars = alt.Chart(hpi_df).mark_bar().encode(
    x='Region:N',
    y='count()',
    color=alt.condition(selected_region, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).properties(
    width=350
).add_params(selected_region)

bars | heatmap + circles

## References

Dash Python User Guide. https://dash.plotly.com/

Dash in 20 Minutes. https://dash.plotly.com/tutorial

Interactive Data Visualization with Python. https://github.com/TrainingByPackt/Interactive-Data-Visualization-with-Python 

PCA Visualization in Python. https://plotly.com/python/pca-visualization/

Plotly Express in Python. https://plotly.com/python/plotly-express/

Plotly Open Source Graphing Library for Python. https://plotly.com/python/

3 Cool Features of Python Altair. https://towardsdatascience.com/3-cool-features-of-python-altair-deb3f432cc11